In [ ]:
#Weights that will suspend rewards for manual review. Optimising for recall.


import pandas as pd
import json
import requests
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from statsmodels.discrete.discrete_model import Logit
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tabulate import tabulate
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict
%matplotlib inline
engine = create_engine(URL(
    account='XXXX',
    region='XXXX',
    user='XX',
    password = "XXX",
    authenticator='EXTERNALBROWSER',
    database='XXX',
    schema='XXX',
    warehouse='XXX'
))
connection = engine.connect()

In [ ]:
tf.reset_default_graph()

In [ ]:
sql_query = """SELECT
*
FROM 
xxxxxx
"""

df = pd.read_sql_query(sql_query, connection)
df = df.rename(columns={'reward_approved':'program_disabled'})
df['program_disabled'] = 1- df.program_disabled

In [ ]:
df.query('program_disabled == 0').head()

In [ ]:
X = df[df.drop(['payout_id','user_id','review_date','program_disabled'], axis=1).columns]
y = df.program_disabled

#log_reg = Logit()

In [ ]:
def convert_to_one_hot_matrix(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [ ]:
X_train_orig, X_test_orig, Y_train_orig, Y_test_orig = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Flatten the training and test sets
x_train = X_train_orig.values.reshape(X_train_orig.shape[0], -1)
x_test = X_test_orig.values.reshape(X_test_orig.shape[0], -1)
# Convert training and test labels to one hot matrices
#y_train = Y_train_orig.values.reshape(X_train_orig.shape[0], -1)
#y_test = Y_test_orig.values.reshape(Y_test_orig.shape[0], -1)
y_train = convert_to_one_hot_matrix(Y_train_orig.values, 2)
y_test = convert_to_one_hot_matrix(Y_test_orig.values, 2)

In [ ]:
m_train = x_train.shape[0]
m_test = x_test.shape[0]
n_input = x_train.shape[1]
n_output = y_train.shape[1]
print("Number of examples in train: " + str(m_train) +
     "\nNumber of examples in test: " + str(m_test) +
     "\nNumber of input features: " + str(n_input) +
     "\nNumber of output features: " + str(n_output))

In [ ]:
#Hyperparameters
learning_rate = 0.0001
training_epochs = 5000
display_step = 500
batch_size = 64
n_hidden_1 = 15
n_hidden_2 = 14
n_hidden_3 = 10

#Constants
print_cost = True

In [ ]:
#Initialise

W1 = tf.get_variable("W1", [n_input,n_hidden_1], dtype=tf.float64, initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable("b1", [n_hidden_1], dtype=tf.float64, initializer = tf.zeros_initializer())
W2 = tf.get_variable("W2", [n_hidden_1,n_hidden_2], dtype=tf.float64, initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable("b2", [n_hidden_2], dtype=tf.float64, initializer = tf.zeros_initializer())
W3 = tf.get_variable("W3", [n_hidden_2,n_hidden_3], dtype=tf.float64, initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable("b3", [n_hidden_3], dtype=tf.float64, initializer = tf.zeros_initializer())
W4 = tf.get_variable("W4", [n_hidden_3,n_output], dtype=tf.float64, initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.get_variable("b4", [n_output], dtype=tf.float64, initializer = tf.zeros_initializer())


keep_prob = tf.placeholder(tf.float64)

In [ ]:
#creating placeholders
x = tf.placeholder(tf.float64, [None,n_input])
y = tf.placeholder(tf.float64)

In [ ]:
#Model
def model(x, W1, b1, W2, b2,W3,b3,W4,b4, keep_prob):
    layer_1 = tf.add(tf.matmul(x, W1), b1)
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, W2), b2)
    layer_2 = tf.nn.relu(layer_2)
    layer_3 = tf.add(tf.matmul(layer_2, W3), b3)
    layer_3 = tf.nn.relu(layer_3)
    logits = tf.add(tf.matmul(layer_3, W4),b4)
    out_layer = tf.sigmoid(logits)
    return out_layer, logits

In [ ]:
predictions, logits = model(x, W1,b1,W2,b2,W3,b3,W4,b4, keep_prob)
cost = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(labels = y, logits = logits, pos_weight = 0.5))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

In [ ]:
costs = []

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
        epoch_cost = 0
        i = 0
        while i < len(x_train):
            start = i
            end = i + batch_size
            batch_x = np.array(x_train[start:end])
            batch_y = np.array(y_train[start:end])

            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                        y: batch_y,keep_prob: 0.8})
            epoch_cost += c/batch_size
            i+=batch_size

            
            # Print the cost every epoch
        if print_cost == True and epoch % display_step == 0:
            print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
        if print_cost == True and epoch % 5 == 0:
            costs.append(epoch_cost)
            
# plot the cost
plt.plot(np.squeeze(costs))
plt.ylabel('cost')
plt.xlabel('iterations (per fives)')
plt.title("Learning rate =" + str(learning_rate))
plt.show()

correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print("Train Accuracy:", accuracy.eval(session = sess,feed_dict = {x: x_train, y: y_train, keep_prob: 1.0}))
print("Test Accuracy:", accuracy.eval(session = sess,feed_dict = {x: x_test, y: y_test, keep_prob: 1.0}))
